In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('./craw_ytn_news_lkh.csv')

In [33]:
df.head()

,title,content,cate,rdate
0,"1단 로켓 고비 넘겼는데, 예상 밖의 복병 3단",[앵커]\r누리호 발사에 가장 큰 변수는 1단 로켓의 정상 작동 여부였습니다.이 고...,3,2021-10-22 02:53
1,"""누리호, 정상 비행은 성공...위성 궤도안착은 실패""","한국형 발사체 누리호가 목표 고도인 700km에는 도달했으나, 탑재체인 '더미 위성...",3,2021-10-22 02:50
2,날씨 변수 뚫고 발사했는데...'누리호' 5번 더 쏜다!,[앵커]\r누리호는 상층에 부는 강한 바람으로 인해 발사 마지막 순간까지 가슴을 졸...,3,2021-10-22 01:53
3,누리호 미완의 도전...발사는 성공·임무는 실패,"[앵커]\r순수 우리나라 기술로 만든 우주 발사체, 누리호가 발사에 성공했습니다.다...",3,2021-10-22 05:33
4,"'뉴 스페이스' 시대, 'K-스페이스' 존재감 알렸다","[앵커]\r누리호의 첫 도전은 비록 완벽한 성공에는 이르지 못했지만, 개발과정에 참...",3,2021-10-22 02:59


In [34]:
df.shape

(1010, 4)

In [35]:
df.isna().sum()

title       0
content    39
cate        0
rdate       0
dtype: int64

In [36]:
isna_idx = df['content'].isna()[df['content'].isna()].index.tolist()

In [37]:
for idx in isna_idx:
    df.drop(index= idx, inplace=True)
print(df.isna().sum())

title      0
content    0
cate       0
rdate      0
dtype: int64


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
tfidf= TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['content'])
print(tfidf_matrix.shape)

(971, 30950)


In [40]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.09619596 0.08227673 ... 0.00799597 0.00726816 0.01467478]
 [0.09619596 1.         0.08424392 ... 0.00397212 0.00399374 0.00390202]
 [0.08227673 0.08424392 1.         ... 0.00646439 0.0057826  0.00749745]
 ...
 [0.00799597 0.00397212 0.00646439 ... 1.         0.86971839 0.75562077]
 [0.00726816 0.00399374 0.0057826  ... 0.86971839 1.         0.79144481]
 [0.01467478 0.00390202 0.00749745 ... 0.75562077 0.79144481 1.        ]]


In [41]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices

title
1단 로켓 고비 넘겼는데, 예상 밖의 복병 3단               0
"누리호, 정상 비행은 성공...위성 궤도안착은 실패"           1
날씨 변수 뚫고 발사했는데...'누리호' 5번 더 쏜다!          2
누리호 미완의 도전...발사는 성공·임무는 실패               3
'뉴 스페이스' 시대, 'K-스페이스' 존재감 알렸다            4
                                      ... 
[날씨] 내일 중부 흐리고 선선...동해안 비             1005
[날씨] 절기 '한로' 전국 비...임실에 가을꽃 만발        1006
[날씨] 절기 '한로' 전국 비...임실에 퍼지는 국화꽃 향기    1007
[날씨] 절기 '한로' 전국 곳곳 비...임실에는 국화꽃 가득    1008
[날씨] 전국 곳곳 비, 한낮 선선...임실에는 국화꽃 만발     1009
Length: 971, dtype: int64

In [42]:
def get_recommendations(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores= sorted(sim_scores, key= lambda x: x[1], reverse=True)
                                    # 기준               True: 내림차순
    sim_scores = sim_scores[1:11]

    news_indices= [i[0] for i in sim_scores]
    return df['title'].iloc[news_indices]

In [44]:
get_recommendations('누리호 미완의 도전...발사는 성공·임무는 실패')

345      '누리호' 미완의 도전...발사는 성공·임무는 실패
352      '누리호' 미완의 도전...발사는 성공·임무는 실패
343     누리호 '절반의 성공'...'위성 안착' 도전은 계속
369         누리호 한 시간 뒤 발사..."최종 점검 중"
365        누리호 이제 우주로...곧 '카운트 다운' 돌입
382        누리호 오늘 우주로...'성공률 30%'에 도전
381    누리호 발사 준비 돌입...발사대 주변 교통 통제 시작
401         누리호 발사대에 도착...우주 향해 우뚝 선다
383          누리호 오늘 우주로...오후 4시 발사 예정
376    "4시 발사 예정 변동 없어"...이 시각 나로우주센터
Name: title, dtype: object